In [1]:
from pyspark.sql import SparkSession


In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
spark = SparkSession.builder.appName("Glue-orchestration").getOrCreate()
print(spark)

25/04/08 15:25:48 WARN Utils: Your hostname, TTNPL-8203 resolves to a loopback address: 127.0.1.1; using 10.1.210.48 instead (on interface wlp0s20f3)
25/04/08 15:25:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/08 15:25:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df = spark.read.csv("file:///home/hdoop/notebooks/data/AWS/Glue-orchestrated/orders.csv",inferSchema = True,header = True)

In [7]:
df.show(n = 7,truncate = False)

+-------+------------+----------+------------+-----------+--------+---------+-----------+---------+
|OrderID|CustomerName|OrderDate |Product     |Category   |Quantity|UnitPrice|TotalAmount|Status   |
+-------+------------+----------+------------+-----------+--------+---------+-----------+---------+
|1001   |John Smith  |2024-03-01|Laptop      |Electronics|1       |1200.0   |1200.0     |Delivered|
|1002   |Alice Brown |2024-03-02|Desk Chair  |Furniture  |2       |150.0    |300.0      |Shipped  |
|1003   |Bob Martin  |2024-03-03|Coffee Maker|Appliances |1       |80.0     |80.0       |Cancelled|
|1004   |Emma Wilson |2024-03-04|Smartphone  |Electronics|2       |650.0    |1300.0     |Delivered|
|1005   |Chris Evans |2024-03-05|Office Desk |Furniture  |1       |400.0    |400.0      |Pending  |
|1006   |Sarah Lee   |2024-03-06|Blender     |Appliances |1       |90.0     |90.0       |Delivered|
|1007   |David Miller|2024-03-07|Monitor     |Electronics|2       |200.0    |400.0      |Delivered|


In [9]:
df_cancelled = df.filter(col("Status") =='Cancelled')

In [10]:
df_cancelled.show()

+-------+-------------+----------+--------------+-----------+--------+---------+-----------+---------+
|OrderID| CustomerName| OrderDate|       Product|   Category|Quantity|UnitPrice|TotalAmount|   Status|
+-------+-------------+----------+--------------+-----------+--------+---------+-----------+---------+
|   1003|   Bob Martin|2024-03-03|  Coffee Maker| Appliances|       1|     80.0|       80.0|Cancelled|
|   1009|  James Scott|2024-03-09|     Microwave| Appliances|       1|    150.0|      150.0|Cancelled|
|   1015|  Ethan Clark|2024-03-15|Vacuum Cleaner| Appliances|       1|    130.0|      130.0|Cancelled|
|   1024|   Ava Torres|2024-03-24|        Heater| Appliances|       1|     75.0|       75.0|Cancelled|
|   1031|  Mason Price|2024-03-31|  Game Console|Electronics|       1|    450.0|      450.0|Cancelled|
|   1041|Carter Hughes|2024-04-10|  Office Chair|  Furniture|       1|    200.0|      200.0|Cancelled|
|   1049|    Owen West|2024-04-18|    Smartphone|Electronics|       1|   

In [17]:
df2 = df.withColumn("Discount",when(col("TotalAmount")>1000,col("TotalAmount")*0.025)\
                   .when(col("TotalAmount")>500,col("TotalAmount")*0.020).otherwise(0))

In [24]:
df2.write.partitionBy("Status").format("csv").option("header","True").mode("append").save("file:///home/hdoop/notebooks/data/AWS/Glue-orchestrated/output/")